In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, balanced_accuracy_score, f1_score, classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_auc_score

In [17]:
df = pd.read_csv('Titanic.csv')
df = df.drop(columns=['Unnamed: 0'])

In [18]:
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,2
1,2,1,1,0,38.0,1,0,71.2833,0
2,3,1,3,0,26.0,0,0,7.9250,2
3,4,1,1,0,35.0,1,0,53.1000,2
4,5,0,3,1,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...,...,...
1304,1305,0,3,1,30.0,0,0,8.0500,2
1305,1306,1,1,0,39.0,0,0,108.9000,0
1306,1307,0,3,1,38.5,0,0,7.2500,2
1307,1308,0,3,1,30.0,0,0,8.0500,2


### Data Splitting Untuk Train & Test
- Train 90% & Test 10%

In [19]:
xtr, xts, ytr, yts = train_test_split(df[['Age','Sex','Pclass','SibSp','Parch','Embarked','Fare']],df['Survived'],test_size=0.1)

# Tree Decision

In [20]:
model = tree.DecisionTreeClassifier()
model.fit(xtr,ytr)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [21]:
print(model.score(xts,yts))

0.8320610687022901


# Parameter Tuning

https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
- Criterion yang Tersedia : Gini & Entropy
- Splitter yang Tersedia : Best & Random

In [34]:
criterion = ['gini','entropy']
splitter = ['best','random']
param = {'criterion':criterion,'splitter':splitter}

# Randomized Search CV

- Mencari Parameter Terbaik

In [35]:
model = tree.DecisionTreeClassifier()
modelrs = RandomizedSearchCV(estimator=model, param_distributions=param, cv=5)

In [36]:
modelrs.fit(xtr,ytr)
print(modelrs.score(xts,yts))

0.8320610687022901


Z:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:281: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


In [37]:
print(modelrs.best_params_)
split = modelrs.best_params_['splitter']
crit = modelrs.best_params_['criterion']

{'splitter': 'best', 'criterion': 'entropy'}


- Setelah Mendapat Parameter Terbaik, Gunakan Parameter Tersebut Supaya Hasil Dari Model Menjadi Maksimal

In [38]:
modelbaru = tree.DecisionTreeClassifier(splitter=split,criterion=crit)
modelbaru.fit(xtr,ytr)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [39]:
predict = modelbaru.predict(df[['Age','Sex','Pclass','SibSp','Parch','Embarked','Fare']])
df['Predict'] = predict
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Predict
0,1,0,3,1,22.0,1,0,7.2500,2,0
1,2,1,1,0,38.0,1,0,71.2833,0,1
2,3,1,3,0,26.0,0,0,7.9250,2,1
3,4,1,1,0,35.0,1,0,53.1000,2,1
4,5,0,3,1,35.0,0,0,8.0500,2,0
...,...,...,...,...,...,...,...,...,...,...
1304,1305,0,3,1,30.0,0,0,8.0500,2,0
1305,1306,1,1,0,39.0,0,0,108.9000,0,1
1306,1307,0,3,1,38.5,0,0,7.2500,2,0
1307,1308,0,3,1,30.0,0,0,8.0500,2,0


In [40]:
modelbaru.score(xts,yts)

0.816793893129771

## Confusion Matrix

- TP = True Positive : Model = 1, Prediksi = 1
- FN = False Negative : Model = 1 , Prediksi = 0
- FP = False Positive = Model = 0 , Prediksi = 1
- TN = True Negative = Model = 0 , Prediksi = 0

In [29]:
tp,fn,fp,tn = confusion_matrix(df['Survived'],df['Predict']).ravel()
tp,fn,fp,tn

(803, 12, 39, 455)

In [30]:
print('Akurasi :',(tp+tn)/(tp+tn+fp+fn))
print('Error Rate :',(fp+fn)/(tp+tn+fp+fn))
print('TP Rate / Recall (+) :',tp/(tp+fn))
print('TN Rate / Recall (-) :',tn/(fp+tn))
print('FP Rate:',fp/(fp+tn))
print('FN Rate :',fn/(tp+fn))
print('Precision (+) :',tp/(tp+fp))
print('Precision (-) :',tn/(tn+fn))
print('Prevalence :',(tp+fn)/(tp+tn+fp+fn))
print('Null Error Rate :',(fp+tn)/(tp+tn+fp+fn))
print('Balance Accuracy Score :',1/2())
Precision = (tp/(tp+fp)) #Precision Positif
Recall = (tp/(tp+fn)) # Recall Positif
print('F1 Score :',2*((Precision*Recall)/(Precision+Recall)))

Akurasi : 0.961038961038961
Error Rate : 0.03896103896103896
TP Rate / Recall (+) : 0.9852760736196319
TN Rate / Recall (-) : 0.9210526315789473
FP Rate: 0.07894736842105263
FN Rate : 0.014723926380368098
Precision (+) : 0.9536817102137767
Precision (-) : 0.974304068522484
Prevalence : 0.6226126814362108
Null Error Rate : 0.37738731856378915
F1 Score : 0.9692214846107422


In [33]:
print('Model Score :',modelbaru.score(df[['Age','Sex','Pclass','SibSp','Parch','Embarked','Fare']],df['Survived']))
print('Accuracy Score :',accuracy_score(df['Survived'],df['Predict']))
print('Recall + :',recall_score(df['Survived'],df['Predict'],pos_label=0))
print('Recall - :',recall_score(df['Survived'],df['Predict'],pos_label=1))
print('Precision + :',precision_score(df['Survived'],df['Predict'],pos_label=0))
print('Precision - :',precision_score(df['Survived'],df['Predict'],pos_label=1))
print('Balanced Accuracy Score :',balanced_accuracy_score(df['Survived'],df['Predict']))
print('F1 Score + :',f1_score(df['Survived'],df['Predict'],pos_label=0))
print('F1 Score - :',f1_score(df['Survived'],df['Predict'],pos_label=1))

Model Score : 0.961038961038961
Accuracy Score : 0.961038961038961
Recall + : 0.9852760736196319
Recall - : 0.9210526315789473
Precision + : 0.9536817102137767
Precision - : 0.974304068522484
Balanced Accuracy Score : 0.9531643525992897
F1 Score + : 0.9692214846107422
F1 Score - : 0.9469302809573361


In [17]:
roc_auc_score(df['Survived'],df['Predict'])

0.9580412806437991

In [18]:
print(classification_report(df['Survived'],df['Predict']))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97       815
           1       0.98      0.93      0.95       494

    accuracy                           0.97      1309
   macro avg       0.97      0.96      0.96      1309
weighted avg       0.97      0.97      0.97      1309

